In [1]:
# 20/02/2023

In [1]:
import pandas as pd
import copy

In [2]:
def combine_tuning_results(tuning_results, output_address):
    """ Combines multiple tuning result DataFrames into one DataFrame"""
    
    assert type(tuning_results) == list
    assert len(tuning_results) > 1

    columns = set(tuning_results[0].columns)
    for tuning_result in tuning_results[1:]:
        assert set(tuning_result.columns) == columns
        

    combined_tuning_results = pd.DataFrame()

    for tuning_result in tuning_results:
        combined_tuning_results = combined_tuning_results.append(tuning_result)
    
    output_address_split = output_address.split('.csv')[0]

    tuning_result.to_csv(f'{output_address_split}.csv', index = False)





class YiLong:



    def __init__(self, type):
        
        # check correct input
        assert type == 'Classification' or type == 'Regression' or 'GLM Regression'

        self.clf_type = type
        self._initialise_objects() # Initialise objects
        print(f'YiLong Initialised to analyse {self.clf_type}')



    def _initialise_objects(self):
        """ Helper to initialise objects """

        self.tuning_result = None
        self.hyperparameters = None
        self._seed = 18861201

        self.regression_extra_output_columns = ['Train r2', 'Val r2', 'Test r2', 
            'Train RMSE', 'Val RMSE', 'Test RMSE', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Time']
        self.classification_extra_output_columns = ['Train accu', 'Val accu', 'Test accu', 
            'Train balanced_accu', 'Val balanced_accu', 'Test balanced_accu', 'Train f1', 'Val f1', 'Test f1', 
            'Train precision', 'Val precision', 'Test precision', 'Train recall', 'Val recall', 'Test recall', 'Time']
        self.GLM_Regression_extra_output_columns = ['Train deviance', 'Val deviance', 'Test deviance', 'Time']

        self.discard_columns = None

        self.tuning_result = None
        self.hyperparameters = None

    

    def read_tuning_result(self, address, extra_to_discard_columns = None):
        """ Read in Tuning Result """

        if extra_to_discard_columns is not None:
            assert type(extra_to_discard_columns) == list

        self.tuning_result = pd.read_csv(address)

        print(f'Successfully read in tuning result, with {len(self.tuning_result)} columns')

        # get list of hyperparameters by taking what is not in the extra_output_columns
        if self.clf_type == 'Classification':
            self.discard_columns = copy.deepcopy(self.classification_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)


        elif self.clf_type == 'Regression':
            self.discard_columns = copy.deepcopy(self.regression_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)


        elif self.clf_type == 'GLM Regression':
            self.discard_columns = copy.deepcopy(self.GLM_Regression_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)

        self.hyperparameters = [col for col in self.tuning_result.columns if col not in self.discard_columns]



    def read_sorted_full_df(self, interested_statistic = None, ascending = False):
        """ View dataframe sorted in reverse in terms of validation score """
        
        assert type(ascending) == bool

        if self.tuning_result is None:
            print('Please run read_tuning_result() first')
            return
        
        if interested_statistic is not None:
            if self.clf_type == 'Regression':
                if interested_statistic not in self.regression_extra_output_columns:
                    print('Statistic not valid for a Regression Model')
                    return

            elif self.clf_type == 'Classification':
                if interested_statistic not in self.classification_extra_output_columns:
                    print('Statistic not valid for a Classification Model')
                    return
            
            elif self.clf_type == 'GLM Regression':
                if interested_statistic not in self.GLM_Regression_extra_output_columns:
                    print('Statistic not valid for a GLM Regression Model')
                    return
        
        

        if len(self.tuning_result) < 60:
            length = len(self.tuning_result)
        else:
            length = 60


        if self.clf_type =='Regression':
            if interested_statistic == None:
                interested_statistic = 'Val r2'

            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)

        elif self.clf_type =='Classification':
            if interested_statistic == None:
                interested_statistic = 'Val accu'
            
            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)
        
        elif self.clf_type =='GLM Regression':
            if interested_statistic == None:
                interested_statistic = 'Val deviance'
            
            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)

        

        sorted_tuning_results.index = range(len(sorted_tuning_results))
        best_hyperparameter_combination = {hyperparameter:sorted_tuning_results.iloc[0][hyperparameter] for hyperparameter in self.hyperparameters}

        # change layout of output
        out_best_hyperparameter_combination = [copy.deepcopy(best_hyperparameter_combination)]
        if 'features' in out_best_hyperparameter_combination[0]:
            out_best_hyperparameter_combination.extend([out_best_hyperparameter_combination[0]['features'], {'feature combo ningxiang score':out_best_hyperparameter_combination[0]['feature combo ningxiang score']}])
            del out_best_hyperparameter_combination[0]['features']
            del out_best_hyperparameter_combination[0]['feature combo ningxiang score']

        print('Best hyperameter combination:', out_best_hyperparameter_combination, '\n')

        print(f'Highest {length}')
        display(sorted_tuning_results.head(length))
        print(f'Lowest {length}')
        display(sorted_tuning_results.tail(length))


        return out_best_hyperparameter_combination 



    def read_mean_val_scores(self):
        """ View the means of evaluation metrics for combinations containing each individual value of a hyperparameter – for each hyperparameter """

        for col in self.hyperparameters: # for each hyperparameter
            
            if col == 'features': # report NingXiang score only
                    continue

            print('\nHYPERPARAMETER:', col.upper())

            hyperparameter_values = list(set(self.tuning_result[col]))
            hyperparameter_values.sort()
            
            # create this temporary dataframe
            validation_score_df = pd.DataFrame()
            for value in hyperparameter_values: # for each value in the hyperparameter

                tmp_df = self.tuning_result[self.tuning_result[col] == value] # select df with only those parameter values

                # get means
                if self.clf_type == 'Classification':
                    tmp_df_mean = tmp_df[self.classification_extra_output_columns[:-1]].mean().T
                elif self.clf_type == 'Regression':
                    tmp_df_mean = tmp_df[self.regression_extra_output_columns[:-1]].mean().T
                elif self.clf_type == 'GLM Regression':
                    tmp_df_mean = tmp_df[self.GLM_Regression_extra_output_columns[:-1]].mean().T

                # get number of observations in this group
                tmp_df_mean['n'] = len(tmp_df)

                # append to this temporary dataframe
                validation_score_df[f'{value}'] = tmp_df_mean

            display(validation_score_df)
    


    def read_grouped_scores(self):
        """ View all evaluation metrics for combinations grouped by containing each individual value of a hyperparameter – for each hyperparameter 
        If any of the individual values of a hyperparameter exceeds 60, then sample down to 60 without replacement """

        for col in self.hyperparameters: # for each hyperparameter

            if col == 'features': # report NingXiang score only
                    continue

            print('\nHYPERPARAMETER:', col.upper())

            hyperparameter_values = list(set(self.tuning_result[col]))
            hyperparameter_values.sort()
            
            # create this temporary dataframe
            for value in hyperparameter_values: # for each value in the hyperparameter

                print(f'{col} Value:', value)
                tmp_df = self.tuning_result[self.tuning_result[col] == value] # select df with only those parameter values

                if len(tmp_df) < 60:
                    length = len(tmp_df)
                else:
                    length = 60

                display(tmp_df.sample(length, replace=False, random_state=self._seed))

In [3]:
yilong = YiLong('Regression')

YiLong Initialised to analyse Regression


In [5]:
yilong.read_tuning_result('../models/tuning/N_xgb_1.csv')

Successfully read in tuning result, with 1350 columns


In [6]:
yilong.read_sorted_full_df()

Best hyperameter combination: [{'gamma': 0.01, 'subsample': 0.75, 'colsample_bytree': 0.75, 'max_depth': 5.0, 'eta': 0.3}] 

Highest 60


,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
0,0.0100,0.75,0.75,5.0,0.30,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.71
1,0.0100,0.75,0.75,5.0,0.15,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.62
2,0.0100,0.75,0.75,5.0,0.45,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.36
3,0.0100,0.75,0.75,5.0,0.75,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,13.79
4,0.0100,0.75,0.75,5.0,0.60,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.80
5,0.0100,0.75,0.75,5.0,0.90,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.70
6,0.0000,0.75,0.75,5.0,0.90,0.6407,0.4948,0.4643,0.1508,0.1789,0.1843,1.230624e+14,1.511062e+14,1.538422e+14,12.67
7,0.0000,0.75,0.75,5.0,0.30,0.6407,0.4948,0.4643,0.1508,0.1789,0.1843,1.230624e+14,1.511062e+14,1.538422e+14,12.35
8,0.0001,0.75,0.75,5.0,0.30,0.6407,0.4948,0.4643,0.1508,0.1789,0.1843,1.230624e+14,1.511061e+14,1.538422e+14,12.70
9,0.0000,0.75,0.75,5.0,0.60,0.6407,0.4948,0.4643,0.1508,0.1789,0.1843,1.230624e+14,1.511062e+14,1.538422e+14,12.33


Lowest 60


,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1290,100.0,0.25,0.75,100.0,0.45,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,39.57
1291,100.0,0.25,0.75,25.0,0.30,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,38.81
1292,100.0,0.25,0.75,50.0,0.60,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,39.07
1293,100.0,0.25,0.75,50.0,0.45,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,39.50
1294,100.0,0.25,0.75,50.0,0.30,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,41.65
1295,100.0,0.25,0.75,10.0,0.45,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,20.29
1296,100.0,0.25,0.75,100.0,0.90,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,39.38
1297,100.0,0.25,0.75,50.0,0.15,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,39.67
1298,100.0,0.25,0.75,25.0,0.60,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,38.16
1299,100.0,0.25,0.75,100.0,0.75,0.1006,0.1013,0.1003,0.2387,0.2386,0.2388,2.686304e+14,2.685729e+14,2.689924e+14,38.97


[{'gamma': 0.01,
  'subsample': 0.75,
  'colsample_bytree': 0.75,
  'max_depth': 5.0,
  'eta': 0.3}]

In [7]:
yilong.read_mean_val_scores()


HYPERPARAMETER: GAMMA


,0.0,0.0001,0.01,1.0,100.0
Train r2,8.856022e-01,8.852844e-01,8.816444e-01,6.937956e-01,1.878667e-01
Val r2,4.418000e-01,4.433578e-01,4.430844e-01,4.705556e-01,1.903778e-01
Test r2,4.115956e-01,4.113844e-01,4.123400e-01,4.397267e-01,1.833556e-01
Train RMSE,6.325111e-02,6.362444e-02,6.961333e-02,1.381711e-01,2.266333e-01
Val RMSE,1.879289e-01,1.876800e-01,1.877156e-01,1.830756e-01,2.262556e-01
Test RMSE,1.930578e-01,1.930956e-01,1.929400e-01,1.884400e-01,2.273889e-01
Train MAPE,4.752275e+13,4.773782e+13,5.114811e+13,1.118825e+14,2.507396e+14
Val MAPE,1.562268e+14,1.562627e+14,1.569543e+14,1.535911e+14,2.506503e+14
Test MAPE,1.589765e+14,1.590577e+14,1.592594e+14,1.565987e+14,2.517654e+14
n,2.700000e+02,2.700000e+02,2.700000e+02,2.700000e+02,2.700000e+02



HYPERPARAMETER: SUBSAMPLE


,0.25,0.5,0.75
Train r2,6.600240e-01,7.196680e-01,7.408240e-01
Val r2,3.643880e-01,4.062613e-01,4.228560e-01
Test r2,3.402080e-01,3.791867e-01,3.956467e-01
Train RMSE,1.299013e-01,1.072187e-01,9.965600e-02
Val RMSE,1.996547e-01,1.932453e-01,1.906933e-01
Test RMSE,2.037333e-01,1.978693e-01,1.953507e-01
Train MAPE,1.161888e+14,9.775161e+13,9.147809e+13
Val MAPE,1.778745e+14,1.742698e+14,1.720667e+14
Test MAPE,1.798593e+14,1.767785e+14,1.747568e+14
n,4.500000e+02,4.500000e+02,4.500000e+02



HYPERPARAMETER: COLSAMPLE_BYTREE


,0.25,0.5,0.75
Train r2,6.998560e-01,7.085893e-01,7.120707e-01
Val r2,3.914800e-01,3.999013e-01,4.021240e-01
Test r2,3.648400e-01,3.759587e-01,3.742427e-01
Train RMSE,1.140867e-01,1.117840e-01,1.109053e-01
Val RMSE,1.955987e-01,1.941893e-01,1.938053e-01
Test RMSE,2.001187e-01,1.982800e-01,1.985547e-01
Train MAPE,1.042548e+14,1.012272e+14,9.993645e+13
Val MAPE,1.794548e+14,1.737565e+14,1.709997e+14
Test MAPE,1.823828e+14,1.755252e+14,1.734866e+14
n,4.500000e+02,4.500000e+02,4.500000e+02



HYPERPARAMETER: MAX_DEPTH


,5.0,10.0,25.0,50.0,100.0
Train r2,5.190289e-01,6.903867e-01,7.670956e-01,7.774756e-01,7.802067e-01
Val r2,4.249044e-01,4.037733e-01,3.894000e-01,3.855622e-01,3.855356e-01
Test r2,3.984022e-01,3.781067e-01,3.630444e-01,3.592733e-01,3.595756e-01
Train RMSE,1.722956e-01,1.287133e-01,9.153778e-02,8.515111e-02,8.359556e-02
Val RMSE,1.899222e-01,1.935600e-01,1.959644e-01,1.966000e-01,1.966089e-01
Test RMSE,1.945556e-01,1.979533e-01,2.004133e-01,2.010244e-01,2.009756e-01
Train MAPE,1.558388e+14,1.127959e+14,8.320226e+13,7.904775e+13,7.814607e+13
Val MAPE,1.739146e+14,1.736396e+14,1.749979e+14,1.755562e+14,1.755768e+14
Test MAPE,1.769439e+14,1.759748e+14,1.774159e+14,1.777253e+14,1.775978e+14
n,2.700000e+02,2.700000e+02,2.700000e+02,2.700000e+02,2.700000e+02



HYPERPARAMETER: ETA


,0.15,0.3,0.45,0.6,0.75,0.9
Train r2,7.068387e-01,7.068387e-01,7.068387e-01,7.068387e-01,7.068387e-01,7.068387e-01
Val r2,3.978351e-01,3.978351e-01,3.978351e-01,3.978351e-01,3.978351e-01,3.978351e-01
Test r2,3.716804e-01,3.716804e-01,3.716804e-01,3.716804e-01,3.716804e-01,3.716804e-01
Train RMSE,1.122587e-01,1.122587e-01,1.122587e-01,1.122587e-01,1.122587e-01,1.122587e-01
Val RMSE,1.945311e-01,1.945311e-01,1.945311e-01,1.945311e-01,1.945311e-01,1.945311e-01
Test RMSE,1.989844e-01,1.989844e-01,1.989844e-01,1.989844e-01,1.989844e-01,1.989844e-01
Train MAPE,1.018062e+14,1.018062e+14,1.018062e+14,1.018062e+14,1.018062e+14,1.018062e+14
Val MAPE,1.747370e+14,1.747370e+14,1.747370e+14,1.747370e+14,1.747370e+14,1.747370e+14
Test MAPE,1.771315e+14,1.771315e+14,1.771315e+14,1.771315e+14,1.771315e+14,1.771315e+14
n,2.250000e+02,2.250000e+02,2.250000e+02,2.250000e+02,2.250000e+02,2.250000e+02


In [8]:
yilong.read_grouped_scores()


HYPERPARAMETER: GAMMA
gamma Value: 0.0


,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
254,0.0,0.50,0.75,100.0,0.45,0.9989,0.4524,0.4255,0.0083,0.1862,0.1908,4.693389e+12,1.497683e+14,1.522318e+14,225.82
1084,0.0,0.25,0.50,5.0,0.75,0.5902,0.4694,0.4472,0.1611,0.1833,0.1872,1.343780e+14,1.572795e+14,1.562228e+14,8.52
946,0.0,0.75,0.25,50.0,0.15,0.9999,0.4246,0.3994,0.0020,0.1909,0.1951,1.010138e+12,1.707113e+14,1.747837e+14,60.95
887,0.0,0.75,0.75,100.0,0.30,1.0000,0.4595,0.4116,0.0017,0.1850,0.1931,8.077842e+11,1.486273e+14,1.544263e+14,198.04
1103,0.0,0.75,0.75,100.0,0.75,1.0000,0.4595,0.4116,0.0017,0.1850,0.1931,8.077842e+11,1.486273e+14,1.544263e+14,201.66
430,0.0,0.50,0.25,50.0,0.75,0.9985,0.4245,0.3829,0.0097,0.1909,0.1978,5.601026e+12,1.661100e+14,1.710778e+14,59.94
502,0.0,0.25,0.75,100.0,0.90,0.9692,0.3866,0.3675,0.0441,0.1971,0.2002,2.824456e+13,1.539239e+14,1.540097e+14,156.61
863,0.0,0.50,0.25,100.0,0.90,0.9988,0.4230,0.3842,0.0087,0.1911,0.1976,5.205012e+12,1.658086e+14,1.698871e+14,77.73
571,0.0,0.50,0.25,50.0,0.30,0.9985,0.4245,0.3829,0.0097,0.1909,0.1978,5.601026e+12,1.661100e+14,1.710778e+14,58.40
356,0.0,0.75,0.75,10.0,0.90,0.9292,0.4591,0.4372,0.0670,0.1851,0.1889,4.455225e+13,1.516942e+14,1.514976e+14,29.85


gamma Value: 0.0001


,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
244,0.0001,0.25,0.50,25.0,0.45,0.9354,0.4055,0.3874,0.0640,0.1940,0.1971,4.015121e+13,1.562091e+14,1.549474e+14,38.89
1093,0.0001,0.25,0.25,100.0,0.15,0.9680,0.3983,0.3570,0.0450,0.1952,0.2019,2.975262e+13,1.598116e+14,1.617472e+14,63.13
1001,0.0001,0.75,0.25,25.0,0.30,0.9994,0.4248,0.3946,0.0061,0.1908,0.1959,2.909183e+12,1.696324e+14,1.744735e+14,35.79
925,0.0001,0.75,0.25,100.0,0.30,0.9998,0.4258,0.4001,0.0031,0.1907,0.1950,1.550898e+12,1.707425e+14,1.741700e+14,71.87
1118,0.0001,0.25,0.25,10.0,0.90,0.7803,0.4334,0.4041,0.1180,0.1894,0.1943,9.001029e+13,1.586130e+14,1.620705e+14,9.86
458,0.0001,0.25,0.75,10.0,0.45,0.7965,0.4373,0.3956,0.1135,0.1888,0.1957,8.323841e+13,1.548018e+14,1.575788e+14,19.77
515,0.0001,0.75,0.50,25.0,0.30,0.9994,0.4561,0.4312,0.0059,0.1856,0.1899,2.857216e+12,1.551432e+14,1.555635e+14,65.63
876,0.0001,0.75,0.75,25.0,0.90,0.9995,0.4600,0.4271,0.0058,0.1849,0.1906,2.715742e+12,1.491120e+14,1.520464e+14,91.19
557,0.0001,0.75,0.25,100.0,0.75,0.9998,0.4258,0.4001,0.0031,0.1907,0.1950,1.550898e+12,1.707425e+14,1.741700e+14,74.42
376,0.0001,0.25,0.25,50.0,0.45,0.9639,0.3899,0.3515,0.0478,0.1965,0.2028,3.134519e+13,1.584567e+14,1.631123e+14,45.01


gamma Value: 0.01


...

,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1327,1.0000,0.75,0.50,100.0,0.6,0.8028,0.4740,0.4479,0.1118,0.1825,0.1871,8.895886e+13,1.518463e+14,1.538727e+14,179.77
1172,100.0000,0.50,0.50,5.0,0.6,0.2108,0.2140,0.2064,0.2236,0.2231,0.2243,2.462455e+14,2.460693e+14,2.472313e+14,9.59
75,0.0001,0.50,0.25,5.0,0.6,0.6024,0.4795,0.4511,0.1587,0.1815,0.1865,1.335663e+14,1.583205e+14,1.614532e+14,6.21
354,0.0100,0.25,0.75,50.0,0.6,0.9566,0.4021,0.3775,0.0524,0.1946,0.1986,3.167548e+13,1.516311e+14,1.527483e+14,94.01
746,0.0100,0.75,0.25,5.0,0.6,0.6140,0.4887,0.4521,0.1563,0.1799,0.1864,1.311649e+14,1.567418e+14,1.597307e+14,5.85
345,0.0001,0.50,0.75,100.0,0.6,0.9988,0.4523,0.4153,0.0087,0.1862,0.1925,4.790751e+12,1.506244e+14,1.533954e+14,225.93
94,1.0000,0.75,0.25,5.0,0.6,0.5685,0.4908,0.4520,0.1653,0.1796,0.1864,1.407031e+14,1.566900e+14,1.604918e+14,6.08
1023,0.0100,0.50,0.50,25.0,0.6,0.9860,0.4506,0.4096,0.0297,0.1865,0.1935,1.661673e+13,1.549963e+14,1.580301e+14,55.98
533,0.0100,0.75,0.50,50.0,0.6,0.9952,0.4532,0.4356,0.0174,0.1861,0.1891,1.034767e+13,1.568359e+14,1.559675e+14,105.34
1307,100.0000,0.25,0.50,25.0,0.6,0.1006,0.1011,0.1003,0.2387,0.2386,0.2388,2.687080e+14,2.686933e+14,2.690300e+14,31.82


eta Value: 0.75


,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1328,0.0000,0.50,0.75,100.0,0.75,0.9989,0.4524,0.4255,0.0083,0.1862,0.1908,4.693389e+12,1.497683e+14,1.522318e+14,230.67
1113,100.0000,0.50,0.25,5.0,0.75,0.1875,0.1905,0.1838,0.2268,0.2264,0.2275,2.515486e+14,2.513582e+14,2.523168e+14,6.32
59,0.0100,0.75,0.50,100.0,0.75,0.9959,0.4531,0.4365,0.0161,0.1861,0.1890,9.689013e+12,1.566930e+14,1.557320e+14,170.82
279,0.0000,0.75,0.50,50.0,0.75,0.9999,0.4541,0.4320,0.0020,0.1859,0.1897,9.225135e+11,1.549472e+14,1.562634e+14,111.20
719,0.0001,0.75,0.50,25.0,0.75,0.9994,0.4561,0.4312,0.0059,0.1856,0.1899,2.857216e+12,1.551432e+14,1.555635e+14,64.09
276,1.0000,0.25,0.25,25.0,0.75,0.6590,0.4524,0.4249,0.1470,0.1862,0.1909,1.187288e+14,1.555908e+14,1.600801e+14,23.38
68,100.0000,0.50,0.75,50.0,0.75,0.2114,0.2131,0.2051,0.2235,0.2232,0.2245,2.461641e+14,2.462888e+14,2.475584e+14,71.85
1019,0.0000,0.50,0.75,50.0,0.75,0.9980,0.4496,0.4152,0.0114,0.1867,0.1925,5.793043e+12,1.500774e+14,1.527406e+14,141.59
474,0.0000,0.75,0.25,25.0,0.75,0.9995,0.4287,0.3968,0.0055,0.1902,0.1955,2.603004e+12,1.697999e+14,1.744198e+14,36.55
1285,100.0000,0.75,0.25,5.0,0.75,0.2562,0.2602,0.2496,0.2170,0.2164,0.2181,2.359074e+14,2.357938e+14,2.372301e+14,6.41


eta Value: 0.9


,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1312,1.0000,0.25,0.25,5.0,0.9,0.5406,0.4774,0.4437,0.1706,0.1819,0.1878,1.458013e+14,1.586365e+14,1.630735e+14,4.96
1138,100.0000,0.75,0.25,100.0,0.9,0.2562,0.2602,0.2496,0.2170,0.2164,0.2181,2.359074e+14,2.357938e+14,2.372301e+14,41.62
50,100.0000,0.25,0.25,50.0,0.9,0.1208,0.1209,0.1208,0.2360,0.2359,0.2361,2.646885e+14,2.647588e+14,2.649878e+14,19.33
292,0.0100,0.75,0.50,100.0,0.9,0.9959,0.4531,0.4365,0.0161,0.1861,0.1890,9.689013e+12,1.566930e+14,1.557320e+14,174.36
715,0.0000,0.75,0.25,100.0,0.9,0.9999,0.4256,0.3976,0.0019,0.1907,0.1954,9.581046e+11,1.703537e+14,1.752758e+14,66.90
291,0.0001,0.50,0.25,25.0,0.9,0.9936,0.4229,0.3951,0.0201,0.1912,0.1958,1.077151e+13,1.672195e+14,1.679942e+14,33.72
64,100.0000,0.25,0.25,100.0,0.9,0.1208,0.1209,0.1208,0.2360,0.2359,0.2361,2.646885e+14,2.647588e+14,2.649878e+14,19.32
1021,1.0000,0.50,0.25,25.0,0.9,0.7225,0.4667,0.4315,0.1326,0.1838,0.1898,1.087589e+14,1.582933e+14,1.626922e+14,29.81
513,100.0000,0.75,0.75,5.0,0.9,0.2501,0.2551,0.2396,0.2179,0.2172,0.2195,2.376643e+14,2.373735e+14,2.396982e+14,12.97
1284,1.0000,0.25,0.50,25.0,0.9,0.6841,0.4490,0.4389,0.1414,0.1868,0.1886,1.123246e+14,1.547658e+14,1.540456e+14,36.19
